In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
import random
from collections import deque
import math

In [0]:
data = pd.read_csv('10percent_sparse_smalldata.csv')
u1_data = data.sort_values(by=['timestamp'])
u1_data

In [0]:
users = u1_data['userId'].values
movies = u1_data['movieId'].values
from sklearn.model_selection import train_test_split

train_users, test_users, train, test = train_test_split(users, u1_data, test_size = 0.25, shuffle = False)

In [0]:
train_users

In [0]:
# Filter out the rating below 3 as we consider high rated items are those rating equal or above 3
indexNames = test[test['rating'] <= 3 ].index
 
# Delete these row indexes from dataFrame
test.drop(indexNames , inplace=True)

In [0]:
test

In [0]:

data_pivoted = train.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
data_pivoted

In [0]:
SEQ_LEN = 5

slide_train_users = [train_users[i: i + SEQ_LEN] for i in range(0, len(train_users) - SEQ_LEN, SEQ_LEN)] #skip until next 5
slide_train_users

In [0]:
tf.reset_default_graph()

LEARNING_RATE = 1e-5
LAYER_SIZE = 256
OUTPUT_SIZE = np.unique(movies).shape[0]
STATE_SIZE = np.unique(users).shape[0]


class Model:
  def __init__(self):
    self.X = tf.placeholder(tf.int32, [None, None])  #initialize placeholder with flexible shape
    self.Y = tf.placeholder(tf.float32, [None, OUTPUT_SIZE])
    self.memory = deque(maxlen = 100)
    
    self.gamma = 0.95
    self.epsilon = 0.1
    self.epsilon_min = 0.01
    self.epsilon_decay = 0.999
        
    embeddings = tf.Variable(tf.random_uniform([STATE_SIZE, LAYER_SIZE], -1, 1)) #random initialize the tensor values
    embedded = tf.nn.embedding_lookup(embeddings, self.X) # batch size, time, dimension
    cell = tf.nn.rnn_cell.LSTMCell(LAYER_SIZE, state_is_tuple = False)
    self.hidden_layer = tf.placeholder(tf.float32, (None, 2 * LAYER_SIZE))
    output, self.last_state = tf.nn.dynamic_rnn(inputs=embedded,cell=cell,
                                                    dtype=tf.float32,
                                                    initial_state=self.hidden_layer)
    self.logits = tf.layers.dense(output[:,-1], OUTPUT_SIZE) #take every output at last index column in "output" according to OUTPUT_SIZE
    self.cost = tf.reduce_sum(tf.square(self.Y - self.logits))
    self.optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(
        self.cost
    )
    self.sess = tf.InteractiveSession()
    self.sess.run(tf.global_variables_initializer())
  
  def act(self, state, memory):
    if random.random() <= self.epsilon:
      return np.random.randint(OUTPUT_SIZE, size = (len(state))), memory
    a, new_memory = self.sess.run([self.logits, self.last_state], 
                                  feed_dict = {self.X: state, self.hidden_layer: memory})
    return np.argmax(a, axis = 1), new_memory
      
  def replay(self, batch_size):
    mini_batch = random.sample(self.memory, batch_size)
    replay_size = len(mini_batch)
    X = []
    Y = np.empty((replay_size, OUTPUT_SIZE))
    MEMORIES = np.empty((replay_size, 2 * LAYER_SIZE)) #why need times 2?
    states = np.array([a[0] for a in mini_batch])
    new_states = np.array([a[3] for a in mini_batch])
    memories = np.array([a[-1] for a in mini_batch])  #state, action, reward, next_state, done
    print(memories)

    Q = self.sess.run(self.logits, feed_dict = {self.X: states, self.hidden_layer: memories})
    Q_new = self.sess.run(self.logits, feed_dict = {self.X: new_states, self.hidden_layer: memories})

    for i in range(len(mini_batch)):
        state, action, reward, next_state, done, rnn_memory = mini_batch[i]
        target = Q[i]
        target[action] = reward
        if not done:
            target[action] += self.gamma * np.amax(Q_new[i])
        X.append(state)
        Y[i] = target
        MEMORIES[i] = rnn_memory
    cost, _ = self.sess.run(
        [self.cost, self.optimizer], feed_dict = {self.X: X, self.Y: Y, self.hidden_layer: MEMORIES}
    )
    if self.epsilon > self.epsilon_min:
        self.epsilon *= self.epsilon_decay
    return cost
  
model = Model()

In [0]:
epoch = 10
batch_size = 256
record = [None] * epoch

for e in range(epoch):
  loss = []
  for i in range(0, len(slide_train_users) - 1, batch_size):
    index = min(i + batch_size, len(slide_train_users) - 1)
    state = slide_train_users[i: index]
    init_value = np.zeros((len(state), 2 * LAYER_SIZE))
    action, init_value = model.act(state, init_value)
    next_state = slide_train_users[i + 1: index + 1]
    
    for no, a in enumerate(action):
      if a >= data_pivoted.shape[1]:
        r = 0
      else:
        r = data_pivoted.loc[slide_train_users[i + no][-1]].iloc[a] # r_prec + r_novelty + r_diversity
      model.memory.append((state[no], a, r, next_state[no], r  < 1, init_value[-1]))
    cost = model.replay(min(batch_size, len(model.memory)))
    loss.append(cost)
    avg_lost = np.mean(loss)
    record[e]= avg_lost

  if e % 1 == 0:
    print('\nepoch: %d, average lost: %f, reward: %f' %(e , avg_lost, r))
    print('----------------------')
    print(state)
    print('----------------------')
    print(action)
    print('----------------------')
    print(next_state)

In [0]:
# Plot graph of Average Lost against Epoch
import matplotlib.pyplot as plt

plt.title("Average lost against Epoch")
plt.plot(epoch)
plt.plot(record)
plt.xlabel('Epoch')
plt.ylabel('Avg_lost')
plt.show()

In [0]:
# aggregate all the movieId that have been rated with above 3 by all user
test_precision = test.copy().groupby('userId')['movieId'].agg(actual = lambda x: list(set(x)))
test_precision

In [0]:
# Function to get the prediction from the model (It similar as the typical library model.predict() function)
def get_users_predictions(user_id, n):
  init_value = np.zeros((1, 2 * LAYER_SIZE))
  p = model.sess.run(model.logits, feed_dict = {model.X: [[user_id]], model.hidden_layer: init_value})[0]
  p = p.argsort()[-n:][::-1]
  return [movies_unique[i] for i in p]

# initialize list to store the prediction output
recs = []

for user in test_precision.index:
  predictions = get_users_predictions(user, 10)
  recs.append(predictions)

In [0]:
test_precision['predictions'] = recs
test_precision

In [0]:
pred_result = pd.DataFrame(test_precision['predictions'])
pred_result

# **Compute Precision**

In [0]:
result = test_precision.apply(lambda x: len(np.intersect1d(x['actual'],x['predictions'])), axis = 1)
result

In [0]:
# Total up the recommendation item which in the high rated item list
intersect = sum(result)
intersect

In [0]:
# user set in test set
U = 353
# length of recommendation list
L = 10

precision = (1 / U) * (intersect/L)
precision

In [0]:
# Test on the first 10 users
first_10 = result[:10]
sum_first_10 = sum(first_10)
sum_first_10

In [0]:
# user set in test set
U = 10
# length of recommendation list
L = 10

precision = (1 / U) * (sum_first_10/L)
precision

# **Compute Novelty**

In [0]:
user_list = data['userId']
user_list = user_list.unique()
user_list

In [0]:
test_novelty = pd.DataFrame(user_list, columns = ["userId"])
test_novelty

In [0]:
novelty_predictions = []

for user in user_list:
  novelty_pred = get_users_predictions(user, 10)
  novelty_predictions.append(novelty_pred)

novelty_predictions

In [0]:
test_novelty['pred_novelty'] = novelty_predictions
test_novelty = test_novelty.set_index('userId')
test_novelty

In [0]:
movie_rated_by_user = data.groupby('movieId')['userId'].count().reset_index(name='userIdCount')
movie_rated_by_user = movie_rated_by_user.set_index('movieId')
movie_rated_by_user

In [0]:
plt.title("Number rated against movieId")
plt.plot(movie_rated_by_user)
plt.xlabel('MovieId')
plt.ylabel('Number of rating')
plt.show()

In [0]:
import math
M = 943
L = 10
sum_novelty = 0

for u in test_novelty.index:
  for i in test_novelty.loc[u]:
    for alpha in i:
      rate_number = movie_rated_by_user.loc[alpha]
      sum_novelty += math.log2(M/rate_number)

novelty = sum_novelty/(M*L)
print(novelty)